In [ ]:
from openai import OpenAI
from os import getenv
from dotenv import load_dotenv
import pandas as pd
import sys
import os
import csv
sys.path.append('..')
from utils.llm_call_utils import perform_experiment

load_dotenv('../config/sergio.env')

In [ ]:
models = {
    "gpt-4o": "openai/gpt-4o", 
    "gpt-4o-mini": "openai/gpt-4o-mini",
    "gpt-4-1": "openai/gpt-4.1",
    "gpt-4-1-mini" : "openai/gpt-4.1-mini",
    "claude-3.7-sonnet": "anthropic/claude-3.7-sonnet",
    "claude-sonnet-4": "anthropic/claude-sonnet-4",
    "gemini-2.5-flash": "google/gemini-2.5-flash",
    "gemini-2.5-pro": "google/gemini-2.5-pro",
    "deepseek-r1-0528": "deepseek/deepseek-r1-0528",
    "deepseek-chat-v3-0324": "deepseek/deepseek-chat-v3-0324",
    "magistral-small-2506": "mistralai/magistral-small-2506",
    "magistral-medium-2506": "mistralai/magistral-medium-2506",
    "llama-3.3-70b-instruct": "meta-llama/llama-3.3-70b-instruct",
    "llama-4-maverick" : "meta-llama/llama-4-maverick",
    "llama-4-scout" : "meta-llama/llama-4-scout",
    "qwen3-14b" : "qwen/qwen3-14b",
    "qwen3-30b-a3b": "qwen/qwen3-30b-a3b",
}

topics = ['immigration']
language = 'English'

In [ ]:
# get the news through the LLM

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("OPENROUTER_GSI_API")
)

for topic in topics:
    print(f'Topic: {topic}')
    perform_experiment(client, models, topic, prompt_filename=f'../config/prompts/{topic}.txt', num_articles=10, days=30, language=language)

In [ ]:
# merge the news by topic

for topic in topics:
    merged = pd.DataFrame()

    for model_name in models:
        filename = f'news/raw/{topic}_{model_name}.csv'
        data = pd.read_csv(filename)
        data['model'] = model_name
        merged = pd.concat([merged, data])

    filename= f'news/merged/{topic}.csv'
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    merged.to_csv(filename, index=False, quoting=csv.QUOTE_ALL)

news articles with pdfs or generic urls must be removed manually